# ISEL - CPS 
## Semestre 2021/22
# <br>
###  <center> Trabalho Prático 2</center>
# </br>

Trabalho realizado por:
* Henrique Matos, numero <b> 48608 </b>
* Tomas Marta, numero <b> 47519 </b>




A figura representa o esquema típico do processo de envio e recepção de informação usando modulações
digitais.

![fig_trab](esquemaCPS.png)

O 2º trabalho foca os blocos Codificação/descodificação do sinal e os blocos Deteção e Correção de erros. O canal é modelado pelo modelo Binary Symmetric Channel ([BSC](https://en.wikipedia.org/wiki/Binary_symmetric_channel)) 

# Codificação
Construa uma função Python que dado um Numpy array (com dimensão $N$) com valores inteiros, retorne um Numpy array (com dimensão $N.R\times 1$) com os mesmos valores convertidos para binário.
Esta função ainda deve receber como parâmetros de entrada o número de bits ($R$) a usar na conversão de cada inteiro.


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
import scipy.io.wavfile as wav

def Codific(arr_signal_quantized,R):
    # resposta        
    sinal = "0" +str(R) +"b"    
    array_sinal = list(map(lambda z: format(z,sinal),arr_signal_quantized))

    binary = itertools.chain(*array_sinal)     
    arr_binary = list(map(int, binary))
    
    return np.array(arr_binary)


Construa uma função que dado um Numpy array com valores os bits, faça a conversão para inteiros (considerando R bits).

In [7]:
def Descodific(arr_binary,R):
    #resposta
    arr_signal = []     
    x = 0;     
    ze = R     
    for i in range(int(len(arr_binary)/R)):                  
        arr_conv = arr_binary[x:ze]                  
        arr_conv = "".join(map(str,arr_conv))                  
        arr_signal.append(arr_conv)                  
        x+=R                  
        ze+=R              
    arr_signal = list(map(lambda z: int(z, 2),arr_signal)) 
    
    return arr_signal


#funcoes auxiliares


def sinal_normalizado(sinal):          
    sinal_normal = sinal/max(sinal, default=0)
             
    return sinal_normal 

def Quantificador(x, Vq, Iq):
    
    xq = np.zeros(len(x))
    iq = np.zeros(len(x))
    
    for i in range (len(x)):
        for j in range (len(Vq)):
            
            if(x[i] < Iq[j]):
                xq[i] = Vq[j]
                iq[i] = j
                break
                
            if(x[i] == Iq[j]):
                xq[i] = Vq[j]
                iq[i] = j
                break
        
    return xq, iq

def Dequantificador(valor,val_quant):          
    x = np.zeros(len(valor)) 

    for i in range(len(valor)):                  
        x[i]=val_quant[valor[i]] 
                     
    return np.array(x) 

def Measure_SNRp(x,y):
    
    Ps = np.sum((x**2.0) / x)
    Pe = np.sum((y**2.0)/ y)
    SNRp = Ps/Pe

    return SNRp

Com base nas funções já implementadas, faça a codificação e descodificação PCM uniforme do sinal de áudio previamente gravado.
Verifique qual a SNR obtida e ouça o sinal descodificado. 
Use para o efeito R=3, 5 e 8.

In [8]:
#Resposta:

fs, y = wav.read("ola.wav")

x = np.array(y[:0],'int16') 

def v_snr(x, R):      
    sinal_normal=sinal_normalizado(x)      
    indices , vq = Quantificador(sinal_normal, R, 0)      
    array_code = Codific(indices, R)      
    array_desc = Descodific(array_code, R)          
    array_dequant = Dequantificador(array_desc,vq)          
    array_desnormal = array_dequant * max(x, default=0)          
    snr = Measure_SNRp(array_desnormal, R)          
    nome = "ola" + str(R) + ".wav"          
    wav.write(nome,fs,array_desnormal.astype("int16"))   
           
    return array_desnormal , snr 

if __name__ == "__main__":                 
    dequant3 , snr3 = v_snr(x, 3) #SNR com R=3        
    dequant5 , snr5 = v_snr(x, 5) #SNR com R=5           
    dequant8 , snr8 = v_snr(x, 8) #SNR com R=8   
                    
    plt.plot(3, snr3, "bo") #SNR com R=3, azul       
    plt.plot(5, snr5, "ro") #SNR com R=5, vermelho       
    plt.plot(8, snr8, "go") #SNR com R=8, verde    

    plt.grid()   
    plt.title("SNR para R = 3, 5 e 8")   
    plt.xlabel("Valores R")     
    plt.ylabel("Valores SNR")          
    plt.show() 

Faça o mesmo teste mas agora com a imagem fornecida.
comente os resultados obtidos.

In [9]:
#Resposta:

# Deteção e Correção de erros

Construa uma função que implemente o Hamming(8,4). Utilize a seguinte matriz geradora:

In [10]:
G=np.array([[1,0,0,0,1,1,0,1],[0,1,0,0,1,0,1,1],[0,0,1,0,0,1,1,1],[0,0,0,1,1,1,1,0]])
G

array([[1, 0, 0, 0, 1, 1, 0, 1],
       [0, 1, 0, 0, 1, 0, 1, 1],
       [0, 0, 1, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 0]])

In [25]:
import numpy as np
import matplotlib.pyplot as plt

#from cps202122_Trab2 import Codific

def Hamming_8_4(arr_bits):
    # resposta

    """ Ik = np.array([[1,0,1],[0,1,1],[1,1,1],[1,1,0]])
    
    
    c = np.array_split(arr_bits, 2)


    c1 = np.dot(c[0], Ik) / 2
    c2 = np.dot(c[1], Ik) / 2

    codigo = np.concatenate((c1,c2), axis=None) """

    bitsmsg = 4
    bitstotal = 8

    bitscontrol = bitstotal-bitsmsg

    p = np.array([[1,1,0,1],
                  [1,0,1,1],
                  [0,1,1,1],
                  [1,1,1,0],
                  [1,0,0,0],
                  [0,1,0,0],
                  [0,0,1,0],
                  [0,0,0,1]])   
    
    identidade = np.eye(bitsmsg)

    g = np.hstack((identidade, p))

    msgatual = arr_bits

    resto = len(msgatual) % bitsmsg

    if(not(resto == 0)):
        sizefinal = len(msgatual) + (bitsmsg-resto)
        msgatual = np.zeros(sizefinal)

        for i in range(len(arr_bits)):
            msgatual[i] = arr_bits[i]

    size = len(msgatual) + ((len(msgatual) / bitsmsg) * bitscontrol)

    bitsfinal = np.zeros(int(size))

    count = 0

    for i in range(0, len(msgatual), bitsmsg):
        current = msgatual[i:i+bitsmsg]
        bcontrol = np.mod(np.dot(current, G), 2)

        for x in range(0, len(bcontrol)):
            bitsfinal[count] = bcontrol[x]
            count += 1


    return bitsfinal.astype("int16")

Construa uma função que dado um Numpy array M bits, para cada 8 bits detecte e corriga possíveis erros, sendo o resultado 4 bits de mensagem.


In [26]:
def Detetor(arr_binary):
    #resposta
    bitsmsg = 4
    bitstotal = 8

    bitscontrol = bitstotal - bitsmsg

    p = np.array([[1,1,0,1],
                  [1,0,1,1],
                  [0,1,1,1],
                  [1,1,1,0],
                  [1,0,0,0],
                  [0,1,0,0],
                  [0,0,1,0],
                  [0,0,0,1]])

    aux = 0
    count = 0
    correctedbits = 0
    size = ((len(arr_binary)/bitstotal) * bitsmsg)
    correctedmsg = np.zeros(int(size))

    for i in range(0, len(arr_binary), bitstotal):
        currentmsg = arr_binary[i: i + bitstotal]
        sindromer = np.mod(np.dot(currentmsg,p),2)

        for x in range(0, len(p)):

            if (sum(p[x] == sindromer) == bitscontrol):
                arr_binary[i+count] = (arr_binary[i+count]+1) % 2
                correctedbits +=1

                break
            else:
                count+=1
        count = 0
        correctedmsg[aux:aux+bitsmsg] = arr_binary[i:i+bitsmsg]

    
    
    
    
    
    return correctedbits, correctedmsg.astype("int16")

Faça um conjunto de testes para verificar o correto funcionamento das funções.

In [ ]:
a = np.array([1,1,1,1,1,1,1,1])

Hamming_8_4(a)

array([1.5, 1.5, 1.5, 1.5, 1.5, 1.5])

Construa uma função que meça a BER entre o array de bits enviado e o array de bits recebido.

In [ ]:
def BER_pratico(arr_bin_tx, arr_bin_rx):
    # resposta
    return BER_pratico

Com as duas funções realizadas e admitindo que os restantes blocos do emissor, canal e receptor possam ser simulados pela expressão $y=1*np.logical\_xor(x,np.random.binomial(1, BER_t,len(x)))$, onde $BER_t$ contém o valor pretendido para o bit error rate.

Meça o BER antes e após a correcção de erros, para diferentes valores de $BER_t$.

In [ ]:
#x - array de bits a entrar no canal
#y - array de bits a sair do canal
import wave
import scipy.io.wavfile as wav
import numpy as np


def sinal_normalizado(sinal):
    sinal_normal = sinal/max(sinal)
    return sinal_normal



def tabela(nbits, vmax, tipo):

    delta = 2*vmax / (2**nbits)

    if(tipo ==0):
        valores_quantificacao = np.arange(-vmax + delta/2, vmax, delta)
        valor_decisao = np.arange(-vmax + delta, vmax - (delta/2), delta)

    elif(tipo ==1):
        valores_quantificacao = np.arange(-vmax+delta, delta, delta)
        valor_decisao = np.arange(-vmax/2+delta/2,delta)
    
    return valores_quantificacao, valor_decisao


def quant(amps, nbits, tipo):
    vmax = np.amax(amps)
    val_tabela = tabela(nbits,vmax,tipo)
    vq = val_tabela

    indices = np.array(list(map(lambda z: np.abs(vq-z).argmin(), amps)))

    array_quant = vq[indices]

    return array_quant,indices

fs, x = wav.read("ola.wav")
data = np.array(x[:, 0], 'int16')
R = 8
BERt = 0.1
sinal_normal = sinal_normalizado(data)
SQ, IQ, VQ = quant(sinal_normal, R, 0)
sinalcod = Codific(IQ,R)
ham = Hamming_8_4(sinalcod)
y = 1 * np.logical_xor(ham, np.random.binomial( 1, BERt, len(ham)))
correctedb, correctedm = Detetor(y)

BerScorrecao = sum(np.logical_xor(ham,y)) / float(len(ham))

print("ber sem correcao")
print(str(BerScorrecao))


#medicao do ber apos correcao

erro = 0
for i in range(len(sinalcod)):
    if(sinalcod[i] != correctedm[i]):
        erro = erro +1


#com correcao

berCorrecao = float(erro) / float(len(sinalcod))

print("BER com correcao")
print(str(berCorrecao))

#resposta

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

Considere o sinal de áudio previamente gravado. Meça SNR na recepção, o BER antes e após a correcção de erros, para diferentes valores de  𝐵𝐸𝑅𝑡 . (considera-se que o ruído é a diferença entre o sinal à entrada do transmissor e o sinal à saída do receptor); Comente os resultados obtidos.

Considere a imagem fornecida e meça a SNR e o BER com e sem correção de erros.
Visualize a imagem no receptor e comente os resultados.

In [27]:
import numpy as np
import scipy.stats
import PIL
from PIL import Image

#SNR

def signaltonoise(a, axis=0, ddof=0):
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, m/sd)

img = Image.open("flower.jpeg")

print(signaltonoise(img, axis=None))




1.9573066979273124
